In [ ]:
import pandas as pd
import numpy as np
import re
import io
import base64
from ipywidgets import VBox, HBox, Layout, Label, Button, Text, Output, FileUpload, HTML, IntText, IntProgress
from IPython.display import display, clear_output, HTML as DisplayHTML

# ==============================================================================
# 1. ESTILIZAÇÃO CSS (Design de Página Web Inteira)
# ==============================================================================
estilo_voila = """
<style>
    body { background-color: #f0f2f5; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; }
    .main-app { 
        max-width: 900px; margin: 40px auto; background: white; 
        padding: 30px; border-radius: 15px; box-shadow: 0 8px 24px rgba(0,0,0,0.1); 
    }
    .web-header { 
        background: linear-gradient(135deg, #0f172a 0%, #2563eb 100%); 
        padding: 30px; border-radius: 12px; color: white; margin-bottom: 30px; text-align: center;
    }
    .card-section { 
        background: #ffffff; border: 1px solid #e2e8f0; border-radius: 10px; 
        padding: 20px; margin-bottom: 20px; transition: transform 0.2s;
    }
    .card-section:hover { border-color: #3b82f6; }
    .download-area { 
        display: flex; flex-wrap: wrap; gap: 10px; margin-top: 20px; 
        padding: 20px; background: #f8fafc; border-radius: 10px; border: 1px dashed #cbd5e1;
    }
    .download-btn { 
        background-color: #059669; color: white !important; padding: 12px 24px; 
        text-decoration: none; border-radius: 8px; font-weight: 600; font-size: 14px;
    }
    .download-btn:hover { background-color: #047857; text-decoration: none; }
    .status-log { font-size: 14px; padding: 10px; border-left: 4px solid #3b82f6; background: #f1f5f9; margin: 5px 0; }
</style>
"""

# ==============================================================================
# 2. CLASSE DE PROCESSAMENTO (Lógica Original Integrada)
# ==============================================================================
class ProcessadorPlanilhas:
    def __init__(self, ciclo, ano):
        self.ciclo = ciclo
        self.ano = ano
        self.nm_ciclo = f"{ano}{ciclo}"

    def _encontrar_aba(self, xl, nomes_possiveis):
        for nome in nomes_possiveis:
            for aba_real in xl.sheet_names:
                if aba_real.strip().lower() == nome.strip().lower():
                    return aba_real
        return None

    def _limpar_dados(self, df):
        df_limpo = df.copy()
        for coluna in df_limpo.columns:
            df_limpo[coluna] = df_limpo[coluna].astype(str)
            df_limpo[coluna] = df_limpo[coluna].apply(
                lambda x: x.replace('.0', '') if re.match(r'^\d+\.0$', x) else x
            )
            df_limpo[coluna] = df_limpo[coluna].replace({
                'nan': '', 'None': '', 'NaN': '', 'NaT': '', '<NA>': '', 'NULL': '', 'null': ''
            })
        return df_limpo

    def _validar_ordem_colunas(self, df, colunas_esperadas):
        for col in colunas_esperadas:
            if col not in df.columns: df[col] = ""
        return df[colunas_esperadas]

    def processar_geral(self, df):
        df_final = df.copy()
        coluna_no = next((col for col in df_final.columns if col.strip().upper() == 'NO'), None)
        if coluna_no: df_final = df_final.rename(columns={coluna_no: 'NO'})
        df_final['NM_CICLO'] = self.nm_ciclo
        cols = ['NO', 'NM_CICLO', 'Avon_Topo_Pontos', 'Avon_Topo_Desconto', 'Avon_Base_Valor', 'Avon_Base_Valor_Centavos', 'Avon_Ganhe', 'Avon_Ganhe_Mecanica', 'Natura_Topo_Pontos', 'Natura_Topo_Desconto', 'Natura_Base_Valor', 'Natura_Base_Valor_Centavo', 'Natura_Ganhe', 'Natura_Ganhe_Mecanica', 'Avon_Link_Cta', 'Natura_Link_Cta']
        return self._limpar_dados(self._validar_ordem_colunas(df_final, cols))

    def processar_avon(self, df):
        df_final = df.copy()
        df_final['NM_CICLO'] = self.nm_ciclo
        # (Simplificado para o exemplo, use suas colunas completas aqui)
        return self._limpar_dados(df_final)

    def processar_natura(self, df):
        df_final = df.copy()
        df_final['NM_CICLO'] = self.nm_ciclo
        return self._limpar_dados(df_final)

    def processar_variaveis(self, df):
        # (Mantenha sua lógica de processamento de variáveis aqui)
        return self._limpar_dados(df)

# ==============================================================================
# 3. INTERFACE E AÇÕES
# ==============================================================================

output_area = Output()
progress_bar = IntProgress(value=0, min=0, max=100, bar_style='info', layout=Layout(width='100%', visibility='hidden'))

ano_widget = IntText(value=2025, layout=Layout(width='100%'))
ciclo_widget = IntText(value=1, layout=Layout(width='100%'))
upload_widget = FileUpload(accept='.xlsx', multiple=False, description='Selecionar Excel (.xlsx)', layout=Layout(width='100%', height='45px'))
process_btn = Button(description='INICIAR PROCESSAMENTO', button_style='primary', icon='play', layout=Layout(width='100%', height='50px'))

def log(msg, tipo="info"):
    cores = {"info": "#3b82f6", "sucesso": "#10b981", "erro": "#ef4444"}
    with output_area:
        display(HTML(f"<div class='status-log' style='border-color: {cores[tipo]}'>{msg}</div>"))

def gerar_download(df, nome):
    csv = df.to_csv(index=False, encoding='utf-8')
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{nome}" class="download-btn">Baixar {nome}</a>'

def acao_principal(b):
    with output_area: clear_output()
    progress_bar.visibility = 'visible'
    progress_bar.value = 10
    
    try:
        if not upload_widget.value:
            log("Erro: Nenhum arquivo detectado!", "erro")
            return

        ano = str(ano_widget.value)
        ciclo = str(ciclo_widget.value).zfill(2)
        
        # Leitura do Upload
        content = list(upload_widget.value.values())[0]['content'] if isinstance(upload_widget.value, dict) else upload_widget.value[0]['content']
        xl = pd.ExcelFile(io.BytesIO(content))
        processor = ProcessadorPlanilhas(ciclo, ano)
        
        progress_bar.value = 40
        log(f"Arquivo carregado. Processando Ciclo {ciclo}/{ano}...", "info")
        
        downloads_html = "<div class='download-area'>"
        abas_map = {'Geral': ['Geral'], 'Avon': ['Avon'], 'Natura': ['Natura'], 'Cards': ['Variáveis', 'Variaveis']}
        
        for chave, nomes in abas_map.items():
            aba_real = processor._encontrar_aba(xl, nomes)
            if aba_real:
                df_raw = xl.parse(aba_real)
                if chave == 'Geral': df_out = processor.processar_geral(df_raw)
                elif chave == 'Avon': df_out = processor.processar_avon(df_raw)
                elif chave == 'Natura': df_out = processor.processar_natura(df_raw)
                else: df_out = processor.processar_variaveis(df_raw)
                
                downloads_html += gerar_download(df_out, f"Nova_Imba_{chave}_C{ciclo}.csv")
                log(f"Aba {chave} processada com sucesso.", "sucesso")
        
        downloads_html += "</div>"
        progress_bar.value = 100
        with output_area: display(HTML(downloads_html))

    except Exception as e:
        log(f"Erro Crítico: {str(e)}", "erro")
        progress_bar.bar_style = 'danger'

process_btn.on_click(acao_principal)

# ==============================================================================
# 4. LAYOUT FINAL (PÁGINA WEB)
# ==============================================================================

app_layout = VBox([
    HTML(estilo_voila),
    VBox([
        HTML("<div class='web-header'><h1>Natura & Avon DataProcessor</h1><p>Versão Web para Imbatíveis Iniciantes</p></div>"),
        HBox([
            VBox([Label("Ano do Ciclo:"), ano_widget], layout=Layout(width='48%', margin='0 2% 0 0')),
            VBox([Label("Número do Ciclo:"), ciclo_widget], layout=Layout(width='48%'))
        ], className='card-section'),
        VBox([
            Label("Upload da Planilha Base:"),
            upload_widget,
            HTML("<small style='color: #64748b;'>Certifique-se que o arquivo possui as abas: Geral, Avon, Natura e Variáveis.</small>")
        ], className='card-section'),
        process_btn,
        progress_bar,
        output_area
    ], className='main-app')
])

display(app_layout)